In [ ]:
# Paso 1: Instalar bibliotecas necesarias
!pip uninstall datasets -y
!pip install datasets --upgrade
!pip install transformers datasets torch huggingface-hub accelerate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# Paso 2: Importar las bibliotecas
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Paso 3: Configurar Hugging Face Access Token
from huggingface_hub import login
login("tu_token_aqui")



# Juez (GPT-2)
print("Cargando modelo del juez (GPT-2)...")
judge_model_name = "openai-community/gpt2"
judge_model = AutoModelForCausalLM.from_pretrained(judge_model_name)
judge_tokenizer = AutoTokenizer.from_pretrained(judge_model_name)

# Jugador 1 (Pythia 160M)
print("Cargando modelo del Jugador 1 (Pythia 160M)...")
player1_model_name = "EleutherAI/pythia-160m"
player1_model = AutoModelForCausalLM.from_pretrained(player1_model_name)
player1_tokenizer = AutoTokenizer.from_pretrained(player1_model_name)

# Jugador 2 (Cerebras-GPT 111M)
print("Cargando modelo del Jugador 2 (Cerebras-GPT 111M)...")
player2_model_name = "cerebras/Cerebras-GPT-111M"
player2_model = AutoModelForCausalLM.from_pretrained(player2_model_name)
player2_tokenizer = AutoTokenizer.from_pretrained(player2_model_name)

# Paso 5: Configurar dispositivo y mover modelos
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
judge_model.to(device)
player1_model.to(device)
player2_model.to(device)

# Paso 6: Confirmar configuración
print("\nModelos cargados exitosamente:")
print(f"- Juez: {judge_model_name}")
print(f"- Jugador 1: {player1_model_name}")
print(f"- Jugador 2: {player2_model_name}")
print("Modelos movidos al dispositivo:", device)


In [ ]:
import random
import json

def generate_board():
    """Genera un tablero aleatorio de tres en raya."""
    symbols = ["X", "O", " "]
    board = [[random.choice(symbols) for _ in range(3)] for _ in range(3)]
    return board

def check_winner(board):
    """Verifica si hay un ganador en el tablero."""
    for row in board:
        if row[0] == row[1] == row[2] and row[0] != " ":
            return row[0]
    for col in range(3):
        if board[0][col] == board[1][col] == board[2][col] and board[0][col] != " ":
            return board[0][col]
    if board[0][0] == board[1][1] == board[2][2] and board[0][0] != " ":
        return board[0][0]
    if board[0][2] == board[1][1] == board[2][0] and board[0][2] != " ":
        return board[0][2]
    return None

def is_full(board):
    """Verifica si el tablero está lleno."""
    return all(cell != " " for row in board for cell in row)

def generate_examples(n=10):
    """Genera ejemplos de jugadas válidas, inválidas y resultados del juego."""
    examples = []
    for _ in range(n):
        board = generate_board()
        winner = check_winner(board)
        if winner:
            output = f"Estado válido. Jugador {winner} gana."
        elif is_full(board):
            output = "Estado válido. El juego termina en empate."
        else:
            output = "Estado válido. No hay ganador aún."

        input_text = f"Tablero actual:\n" + "\n".join([" | ".join(row) for row in board])
        examples.append({"input": input_text, "output": output})

    return examples

# Generar ejemplos y guardarlos en JSON
examples = generate_examples(100)
with open("tictactoe_examples.json", "w") as f:
    json.dump(examples, f, indent=4)

print("Ejemplos generados y guardados en 'tictactoe_examples.json'")


Ejemplos generados y guardados en 'tictactoe_examples.json'


In [ ]:
# Importar las bibliotecas necesarias
from datasets import load_dataset
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForCausalLM

# Cargar el tokenizador y el modelo preentrenado
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
judge_model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

# Asignar el eos_token como pad_token y configurar el padding
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

# Cargar los datos de entrenamiento y evaluación desde los archivos JSON
train_dataset = load_dataset("json", data_files="tictactoe_examples.json")
eval_dataset = load_dataset("json", data_files="tictactoe_eval.json")

# Definir la función de preprocesamiento para tokenizar las entradas y salidas
def preprocess(examples):
    inputs = examples["input"]
    outputs = examples["output"]
    model_inputs = tokenizer(inputs, truncation=True, padding="max_length", max_length=128)
    labels = tokenizer(outputs, truncation=True, padding="max_length", max_length=128)["input_ids"]
    model_inputs["labels"] = labels
    return model_inputs

# Aplicar el preprocesamiento a los conjuntos de datos
tokenized_train_dataset = train_dataset["train"].map(preprocess, batched=True)
tokenized_eval_dataset = eval_dataset["train"].map(preprocess, batched=True)

# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./tictactoe_judge",
    evaluation_strategy="steps",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    save_steps=500,
    logging_steps=10,
    eval_steps=100  # Frecuencia de evaluación durante el entrenamiento
)

# Crear el entrenador
trainer = Trainer(
    model=judge_model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer
)

# Iniciar el entrenamiento
trainer.train()

# Guardar el modelo entrenado
trainer.save_model("./tictactoe_judge_model")


In [ ]:
"""
Evaluación de Jugadas con el Juez

"""
from transformers import AutoTokenizer, AutoModelForCausalLM
import random
import json

# Cargar el modelo y el tokenizador del juez entrenado
judge_model = AutoModelForCausalLM.from_pretrained("./tictactoe_judge_model")
judge_tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")

# Configurar el token de padding
judge_tokenizer.pad_token = judge_tokenizer.eos_token
judge_tokenizer.padding_side = "left"

# Función para inicializar un tablero vacío
def inicializar_tablero():
    return [[" " for _ in range(3)] for _ in range(3)]

# Función para representar el tablero como texto
def representar_tablero(tablero):
    return "\n".join([" | ".join(fila) for fila in tablero])

# Función para verificar si hay un ganador
def verificar_ganador(tablero):
    for fila in tablero:
        if fila[0] == fila[1] == fila[2] and fila[0] != " ":
            return fila[0]

    for col in range(3):
        if tablero[0][col] == tablero[1][col] == tablero[2][col] and tablero[0][col] != " ":
            return tablero[0][col]

    if tablero[0][0] == tablero[1][1] == tablero[2][2] and tablero[0][0] != " ":
        return tablero[0][0]

    if tablero[0][2] == tablero[1][1] == tablero[2][0] and tablero[0][2] != " ":
        return tablero[0][2]

    return None

# Función para verificar si el tablero está lleno
def tablero_lleno(tablero):
    return all(cell != " " for fila in tablero for cell in fila)

# Función para jugar una partida completa
def jugar_partida():
    tablero = inicializar_tablero()
    turno = random.choice(["X", "O"])
    partida = []

    while True:
        movimiento = random.randint(1, 9)
        fila, col = divmod(movimiento - 1, 3)

        if tablero[fila][col] == " ":
            tablero[fila][col] = turno
            entrada = f"Tablero:\n{representar_tablero(tablero)}\nTurno: {turno}\nMovimiento: {movimiento}"
            salida = ""

            # Verificar estado del juego
            ganador = verificar_ganador(tablero)
            if ganador:
                salida = f"Jugada válida. Resultado: Victoria de {ganador}."
                partida.append({"input": entrada, "expected_output": salida})
                break
            elif tablero_lleno(tablero):
                salida = "Jugada válida. Resultado: Empate."
                partida.append({"input": entrada, "expected_output": salida})
                break
            else:
                salida = "Jugada válida. No hay ganador aún."

            partida.append({"input": entrada, "expected_output": salida})

            # Cambiar turno
            turno = "O" if turno == "X" else "X"

    return partida

# Generar 100 partidas completas
def generar_partidas(n=100):
    todas_las_partidas = []
    for _ in range(n):
        partida = jugar_partida()
        todas_las_partidas.extend(partida)
    return todas_las_partidas

# Evaluar las jugadas con el juez
def evaluar_con_juez(partidas):
    resultados = []
    for jugada in partidas:
        input_text = jugada["input"]

        # Tokenizar la entrada
        inputs_tokenized = judge_tokenizer(
            input_text, truncation=True, padding="max_length", max_length=128, return_tensors="pt"
        )

        # Generar salida del juez
        outputs = judge_model.generate(
            **inputs_tokenized,
            max_new_tokens=50,
            pad_token_id=judge_tokenizer.pad_token_id
        )
        generated_output = judge_tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Guardar el resultado
        resultados.append({
            "input": input_text,
            "expected_output": jugada["expected_output"],
            "generated_output": generated_output,
            "correct": jugada["expected_output"] == generated_output
        })
    return resultados

# Generar y evaluar 100 partidas
partidas = generar_partidas(100)
resultados = evaluar_con_juez(partidas)


# Guardar los resultados en un archivo JSON
with open("evaluated_jugadas_100.json", "w") as f:
    json.dump(resultados, f, indent=4)
    print("Resultados de 100 partidas guardados en 'evaluated_jugadas_100.json'")



Resultados de 100 partidas guardados en 'evaluated_jugadas_100.json'


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import json
import torch
from torch.utils.data import Dataset

# Cargar los modelos y tokenizadores de los jugadores
player1_model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-160m")
player1_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-160m")
player1_tokenizer.pad_token = player1_tokenizer.eos_token
player1_tokenizer.padding_side = "left"

player2_model = AutoModelForCausalLM.from_pretrained("cerebras/Cerebras-GPT-111M")
player2_tokenizer = AutoTokenizer.from_pretrained("cerebras/Cerebras-GPT-111M")
player2_tokenizer.pad_token = player2_tokenizer.eos_token
player2_tokenizer.padding_side = "left"

# Cargar las jugadas evaluadas
with open("evaluated_jugadas_100.json", "r") as f:
    evaluated_data = json.load(f)

# Seleccionar una muestra representativa
max_jugadas = 50  # Limitar a 50 jugadas para cada jugador
jugadas_muestra = evaluated_data[:max_jugadas]

# Crear un dataset personalizado para Hugging Face
class TictactoeDataset(Dataset):
    def __init__(self, jugadas, tokenizer):
        self.examples = []
        for jugada in jugadas:
            inputs = tokenizer(
                jugada["input"], truncation=True, padding="max_length", max_length=128, return_tensors="pt"
            )
            labels = tokenizer(
                jugada["expected_output"], truncation=True, padding="max_length", max_length=128, return_tensors="pt"
            )["input_ids"]
            inputs["labels"] = labels
            self.examples.append(inputs)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return {key: val.squeeze(0) for key, val in self.examples[idx].items()}

# Crear datasets para los jugadores
player1_dataset = TictactoeDataset(jugadas_muestra, player1_tokenizer)
player2_dataset = TictactoeDataset(jugadas_muestra, player2_tokenizer)

# Configurar argumentos de entrenamiento
training_args_player1 = TrainingArguments(
    output_dir="./trained_player1_model",
    learning_rate=5e-5,
    per_device_train_batch_size=2,  # Tamaño de lote reducido
    max_steps=200,  # Limitar a 200 pasos de entrenamiento
    save_steps=100,  # Guardar el modelo cada 100 pasos
    logging_steps=50,  # Registrar información cada 50 pasos
    evaluation_strategy="no"  # Sin evaluación durante el entrenamiento
)

training_args_player2 = TrainingArguments(
    output_dir="./trained_player2_model",
    learning_rate=5e-5,
    per_device_train_batch_size=2,  # Tamaño de lote reducido
    max_steps=200,  # Limitar a 200 pasos de entrenamiento
    save_steps=100,  # Guardar el modelo cada 100 pasos
    logging_steps=50,  # Registrar información cada 50 pasos
    evaluation_strategy="no"  # Sin evaluación durante el entrenamiento
)

# Crear entrenadores para los jugadores
trainer_player1 = Trainer(
    model=player1_model,
    args=training_args_player1,
    train_dataset=player1_dataset,
    tokenizer=player1_tokenizer,
)

trainer_player2 = Trainer(
    model=player2_model,
    args=training_args_player2,
    train_dataset=player2_dataset,
    tokenizer=player2_tokenizer,
)

# Entrenar a Jugador 1
print("Entrenando Jugador 1...")
trainer_player1.train()
trainer_player1.save_model("./trained_player1_model")
print("Jugador 1 entrenado y guardado.")

# Entrenar a Jugador 2
print("Entrenando Jugador 2...")
trainer_player2.train()
trainer_player2.save_model("./trained_player2_model")
print("Jugador 2 entrenado y guardado.")


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-6-ef213f9f3908>:71: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_player1 = Trainer(
max_steps is given, it will override any value given in num_train_epochs
<ipython-input-6-ef213f9f3908>:78: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_player2 = Trainer(
max_steps is given, it will override any value given in num_train_epochs


Entrenando Jugador 1...


Step,Training Loss
50,0.342400
100,0.053600
150,0.037900
200,0.024200


Jugador 1 entrenado y guardado.
Entrenando Jugador 2...


Step,Training Loss
50,0.517100
100,0.105200
150,0.087500
200,0.048900


Jugador 2 entrenado y guardado.



 MODELO Pythia 160M (Jugador 1, X) hizo el movimiento 9
[' ', ' ', ' ']
[' ', ' ', ' ']
[' ', ' ', 'X']

 MODELO Cerebras-GPT 111M (Jugador 2, O) hizo el movimiento 7
[' ', ' ', ' ']
[' ', ' ', ' ']
['O', ' ', 'X']

 MODELO Pythia 160M (Jugador 1, X) hizo el movimiento 2
[' ', 'X', ' ']
[' ', ' ', ' ']
['O', ' ', 'X']

 MODELO Cerebras-GPT 111M (Jugador 2, O) hizo el movimiento 5
[' ', 'X', ' ']
[' ', 'O', ' ']
['O', ' ', 'X']

 MODELO Pythia 160M (Jugador 1, X) hizo el movimiento 8
[' ', 'X', ' ']
[' ', 'O', ' ']
['O', 'X', 'X']

 MODELO Cerebras-GPT 111M (Jugador 2, O) hizo el movimiento 4
[' ', 'X', ' ']
['O', 'O', ' ']
['O', 'X', 'X']

 MODELO Pythia 160M (Jugador 1, X) hizo el movimiento 1
['X', 'X', ' ']
['O', 'O', ' ']
['O', 'X', 'X']

 MODELO Cerebras-GPT 111M (Jugador 2, O) hizo el movimiento 3
['X', 'X', 'O']
['O', 'O', ' ']
['O', 'X', 'X']

¡Victoria de MODELO Cerebras-GPT 111M (O)!
Resultado final: O


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import random
from transformers import AutoTokenizer, AutoModelForCausalLM

# Cargar los modelos y tokenizadores de los jugadores
player1_model = AutoModelForCausalLM.from_pretrained("./trained_player1_model")
player1_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-160m")
player1_tokenizer.pad_token = player1_tokenizer.eos_token

player2_model = AutoModelForCausalLM.from_pretrained("./trained_player2_model")
player2_tokenizer = AutoTokenizer.from_pretrained("cerebras/Cerebras-GPT-111M")
player2_tokenizer.pad_token = player2_tokenizer.eos_token

# Función para inicializar un tablero vacío
def inicializar_tablero():
    return [[" " for _ in range(3)] for _ in range(3)]

# Función para representar el tablero como texto
def representar_tablero(tablero):
    return "\n".join([" | ".join(fila) for fila in tablero])

# Función para verificar si hay un ganador
def verificar_ganador(tablero):
    for fila in tablero:
        if fila[0] == fila[1] == fila[2] and fila[0] != " ":
            return fila[0]

    for col in range(3):
        if tablero[0][col] == tablero[1][col] == tablero[2][col] and tablero[0][col] != " ":
            return tablero[0][col]

    if tablero[0][0] == tablero[1][1] == tablero[2][2] and tablero[0][0] != " ":
        return tablero[0][0]

    if tablero[0][2] == tablero[1][1] == tablero[2][0] and tablero[0][2] != " ":
        return tablero[0][2]

    return None

# Función para verificar si el tablero está lleno
def tablero_lleno(tablero):
    return all(cell != " " for fila in tablero for cell in fila)

# Función para seleccionar un movimiento válido aleatorio
def seleccionar_movimiento_valido(tablero):
    movimientos_validos = [
        (fila, col) for fila in range(3) for col in range(3) if tablero[fila][col] == " "
    ]
    if movimientos_validos:
        fila, col = random.choice(movimientos_validos)
        return fila * 3 + col + 1  # Convertir a movimiento en el rango [1-9]
    return None

# Función para que un jugador haga un movimiento
def realizar_jugada(model, tokenizer, tablero, turno, max_reintentos=3):
    # Generar el texto del estado actual del juego
    tablero_texto = representar_tablero(tablero)
    input_text = f"Tablero:\n{tablero_texto}\nTurno: {turno}\n"

    for _ in range(max_reintentos):
        # Tokenizar y generar la jugada
        inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
        output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.pad_token_id)
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

        # Extraer el movimiento de la salida generada
        try:
            movimiento = int(generated_text.split("Movimiento: ")[1].strip())
            fila, col = divmod(movimiento - 1, 3)
            if 1 <= movimiento <= 9 and tablero[fila][col] == " ":
                return movimiento, True  # Movimiento válido
        except (IndexError, ValueError):
            pass  # Movimiento inválido o error de generación

    # Si todos los reintentos fallan, seleccionar un movimiento válido aleatorio
    movimiento = seleccionar_movimiento_valido(tablero)
    return movimiento, movimiento is not None

# Función para que el juez comente sobre el movimiento
def comentario_del_juez(tablero, turno, movimiento):
    fila, col = divmod(movimiento - 1, 3)
    if 1 <= movimiento <= 9 and tablero[fila][col] == " ":
        return f"COMENTARIO JUEZ: El movimiento {movimiento} por el jugador '{turno}' es válido."
    else:
        return f"COMENTARIO JUEZ: El movimiento {movimiento} por el jugador '{turno}' es inválido."

# Función para determinar las posiciones ganadoras
def obtener_posiciones_ganadoras(tablero):
    for i in range(3):
        # Verificar filas
        if tablero[i][0] == tablero[i][1] == tablero[i][2] and tablero[i][0] != " ":
            return [(i, 0), (i, 1), (i, 2)]
        # Verificar columnas
        if tablero[0][i] == tablero[1][i] == tablero[2][i] and tablero[0][i] != " ":
            return [(0, i), (1, i), (2, i)]
    # Verificar diagonales
    if tablero[0][0] == tablero[1][1] == tablero[2][2] and tablero[0][0] != " ":
        return [(0, 0), (1, 1), (2, 2)]
    if tablero[0][2] == tablero[1][1] == tablero[2][0] and tablero[0][2] != " ":
        return [(0, 2), (1, 1), (2, 0)]
    return []

# Función para mostrar el tablero resaltando las posiciones ganadoras
def mostrar_tablero_con_victoria(tablero, posiciones_ganadoras):
    tablero_resaltado = [[cell for cell in row] for row in tablero]
    for fila, col in posiciones_ganadoras:
        tablero_resaltado[fila][col] = f"*{tablero_resaltado[fila][col]}*"
    return "\n".join([" | ".join(fila) for fila in tablero_resaltado])

# Función para jugar una partida completa
def jugar_partida():
    tablero = inicializar_tablero()
    turno = "X"  # Comienza el Jugador 1
    jugador_actual = 1

    while True:
        # Seleccionar el modelo y tokenizador del jugador actual
        if jugador_actual == 1:
            model = player1_model
            tokenizer = player1_tokenizer
            modelo_nombre = "Pythia 160M"
        else:
            model = player2_model
            tokenizer = player2_tokenizer
            modelo_nombre = "Cerebras-GPT 111M"

        # Realizar la jugada
        movimiento, valido = realizar_jugada(model, tokenizer, tablero, turno)

        # Comentario del juez
        comentario = comentario_del_juez(tablero, turno, movimiento)

        if not valido:
            print(f"Movimiento inválido por MODELO {modelo_nombre} (Jugador {jugador_actual}). Turno perdido.")
            print(comentario)
            jugador_actual = 3 - jugador_actual  # Cambiar de jugador
            turno = "O" if turno == "X" else "X"
            continue

        # Actualizar el tablero
        fila, col = divmod(movimiento - 1, 3)
        tablero[fila][col] = turno

        # Mostrar el estado actual del tablero
        print(f"\nMODELO {modelo_nombre} (Jugador {jugador_actual}, {turno}) hizo el movimiento {movimiento}")
        for fila in tablero:
            print(fila)
        print(comentario)

        # Verificar estado del juego
        ganador = verificar_ganador(tablero)
        if ganador:
            posiciones_ganadoras = obtener_posiciones_ganadoras(tablero)
            modelo_ganador = "Pythia 160M" if ganador == "X" else "Cerebras-GPT 111M"
            print(f"\n¡Victoria de MODELO {modelo_ganador} ({ganador})!")
            print(f"COMENTARIO JUEZ: ¡El jugador '{ganador}' con el modelo {modelo_ganador} ganó!")
            print("Posiciones ganadoras:")
            print(mostrar_tablero_con_victoria(tablero, posiciones_ganadoras))
            return ganador

        if tablero_lleno(tablero):
            print("\n¡Empate!")
            return "Empate"

        # Cambiar de jugador
        jugador_actual = 3 - jugador_actual
        turno = "O" if turno == "X" else "X"

# Jugar una partida completa
resultado = jugar_partida()
print(f"Resultado final: {resultado}")



MODELO Pythia 160M (Jugador 1, X) hizo el movimiento 7
[' ', ' ', ' ']
[' ', ' ', ' ']
['X', ' ', ' ']
COMENTARIO JUEZ: El movimiento 7 por el jugador 'X' es válido.

MODELO Cerebras-GPT 111M (Jugador 2, O) hizo el movimiento 1
['O', ' ', ' ']
[' ', ' ', ' ']
['X', ' ', ' ']
COMENTARIO JUEZ: El movimiento 1 por el jugador 'O' es válido.

MODELO Pythia 160M (Jugador 1, X) hizo el movimiento 2
['O', 'X', ' ']
[' ', ' ', ' ']
['X', ' ', ' ']
COMENTARIO JUEZ: El movimiento 2 por el jugador 'X' es válido.

MODELO Cerebras-GPT 111M (Jugador 2, O) hizo el movimiento 3
['O', 'X', 'O']
[' ', ' ', ' ']
['X', ' ', ' ']
COMENTARIO JUEZ: El movimiento 3 por el jugador 'O' es válido.

MODELO Pythia 160M (Jugador 1, X) hizo el movimiento 5
['O', 'X', 'O']
[' ', 'X', ' ']
['X', ' ', ' ']
COMENTARIO JUEZ: El movimiento 5 por el jugador 'X' es válido.

MODELO Cerebras-GPT 111M (Jugador 2, O) hizo el movimiento 6
['O', 'X', 'O']
[' ', 'X', 'O']
['X', ' ', ' ']
COMENTARIO JUEZ: El movimiento 6 por el jug